In [ ]:
!rm -rf /kaggle/working/ds201_BTTH2
!git clone https://github.com/UngHoangLong/ds201_BTTH2.git

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

import sys
sys.path.append("/kaggle/working/ds201_BTTH2")

from data_loader import get_vinfood_dataloaders
from module import ResNet18
from tqdm.auto import tqdm

In [3]:
train_data_path = "/kaggle/input/vinafood21/VinaFood21/train"
test_data_path = "/kaggle/input/vinafood21/VinaFood21/test"

In [4]:
BATCH_SIZE = 16
RESNET18_IMG_SIZE = 224
train_loader, val_loader, test_loader, NUM_CLASSES = get_vinfood_dataloaders(batch_size=BATCH_SIZE, train_path=train_data_path, test_path=test_data_path, val_split=0.2, image_size=RESNET18_IMG_SIZE)

Đã tải data thành công. Tổng cộng 21 lớp.


In [5]:
# Cấu hình
LEARNING_RATE = 0.001 # theo tiêu chuẩn của Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Khởi tạo model với đúng số lớp
model = ResNet18(num_classes=NUM_CLASSES).to(device)

# Hàm loss
criterion = nn.CrossEntropyLoss()

# Optimizer Adam theo yêu cầu
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

print("Đã khởi tạo model, criterion, và optimizer.")
print(model)

Sử dụng thiết bị: cuda
Đã khởi tạo model, criterion, và optimizer.
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, e

In [6]:
NUM_EPOCHS = 50 # Số epochs bạn muốn chạy
history = {'train_loss': [], 'val_f1': [], 'val_precision': [], 'val_recall': []}

# --- THÊM MỚI: Biến để theo dõi model tốt nhất ---
best_val_f1 = 0.0  # Bắt đầu với F1 = 0
MODEL_SAVE_PATH = "best_resnet18_model.pth" # Tên file để lưu model
# -----------------------------------------------

print('Bắt đầu quá trình huấn luyện')
for epoch in range(NUM_EPOCHS):
    
    # --- Training ---
    model.train()
    running_loss = 0.0 # biến này lưu trữ loss của từng epoch
    
    train_progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} - Train', leave=False)
    
    for images, labels in train_progress_bar: # Lặp qua progress bar mới
        # print('bắt đầu batch') # Bạn có thể bỏ comment này nếu muốn xem chi tiết
        images, labels = images.to(device), labels.to(device)

        output = model(images) # chạy def forward
        loss = criterion(output, labels) #

        optimizer.zero_grad() # xoá grad của batch cũ
        loss.backward() # tính toán gradient (tìm đường xuống dốc)
        optimizer.step() # Cập nhật trọng số (bước xuống dốc)
        running_loss += loss.item() # 
        
        train_progress_bar.set_postfix(batch_loss=loss.item())

    epoch_train_loss = running_loss / len(train_loader) # tổng loss chia cho tổng số batch
    history['train_loss'].append(epoch_train_loss)

    # --- Validation ---
    model.eval()
    val_preds=[]
    val_labels=[]
    
    val_progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} - Val', leave=False)
    
    with torch.no_grad(): # không theo dõi hay tính gradient gì 
        for images, labels in val_progress_bar: # Lặp qua progress bar mới
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, predicted = torch.max(output.data, 1) # không cần softmax vì logits lớn thì xác xuất cũng lớn

            val_preds.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    # Tính metrics trên tập Validation
    precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)
    
    history['val_precision'].append(precision)
    history['val_recall'].append(recall)
    history['val_f1'].append(f1)
    
    # Print kết quả của tập Validation
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {epoch_train_loss:.4f} | Val F1 (Macro): {f1:.4f}")

    # --- THÊM MỚI: Logic lưu model ---
    if f1 > best_val_f1:
        best_val_f1 = f1 # Cập nhật F1 tốt nhất
        # Lưu lại "trạng thái" (state_dict) của model
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"  -> Đã lưu model tốt nhất mới! F1: {best_val_f1:.4f} tại {MODEL_SAVE_PATH}")
    # ------------------------------------

print(f"\nHuấn luyện hoàn tất! F1 tốt nhất trên Validation là: {best_val_f1:.4f}")

Bắt đầu quá trình huấn luyện


Epoch 1/50 - Train:   0%|          | 0/503 [00:00<?, ?it/s]

Epoch 1/50 - Val:   0%|          | 0/126 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/50 - Train Loss: 2.8148 | Val F1 (Macro): 0.1438
  -> Đã lưu model tốt nhất mới! F1: 0.1438 tại best_resnet18_model.pth


...

Epoch 49/50 - Val:   0%|          | 0/126 [00:00<?, ?it/s]

Epoch 49/50 - Train Loss: 0.0347 | Val F1 (Macro): 0.5216


Epoch 50/50 - Train:   0%|          | 0/503 [00:00<?, ?it/s]

Epoch 50/50 - Val:   0%|          | 0/126 [00:00<?, ?it/s]

Epoch 50/50 - Train Loss: 0.0278 | Val F1 (Macro): 0.5073

Huấn luyện hoàn tất! F1 tốt nhất trên Validation là: 0.5390


In [7]:
# (Cell 5 - Đánh giá cuối cùng trên tập Test)

print("Đang đánh giá kết quả cuối cùng trên tập Test...")

# 1. Khởi tạo lại kiến trúc model (phải giống hệt)
# Đảm bảo `LeNet` và `NUM_CLASSES` đã được định nghĩa ở các cell trên
test_model = ResNet18(num_classes=NUM_CLASSES).to(device)

# 2. Tải trọng số (weights) đã lưu từ file
test_model.load_state_dict(torch.load(MODEL_SAVE_PATH))

# 3. Chuyển sang chế độ đánh giá
test_model.eval()

test_preds = []
test_labels = []

# Thêm tqdm cho vòng lặp test
test_progress_bar = tqdm(test_loader, desc="Testing", leave=False)

with torch.no_grad(): # không theo dõi hay tính gradient gì 
    for images, labels in test_progress_bar:
        images, labels = images.to(device), labels.to(device)
        
        # Dùng test_model (đã được tải) để dự đoán
        outputs = test_model(images)
        
        _, predicted = torch.max(outputs.data, 1) # không cần softmax
        
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Tính toán các độ đo cuối cùng
test_precision = precision_score(test_labels, test_preds, average='macro', zero_division=0)
test_recall = recall_score(test_labels, test_preds, average='macro', zero_division=0)
test_f1 = f1_score(test_labels, test_preds, average='macro', zero_division=0)

print("\n--- KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST (từ model tốt nhất) ---")
print(f"Test Precision (Macro): {test_precision:.4f}")
print(f"Test Recall (Macro):    {test_recall:.4f}")
print(f"Test F1-Score (Macro):  {test_f1:.4f}")

Đang đánh giá kết quả cuối cùng trên tập Test...


Testing:   0%|          | 0/418 [00:00<?, ?it/s]


--- KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST (từ model tốt nhất) ---
Test Precision (Macro): 0.4908
Test Recall (Macro):    0.4584
Test F1-Score (Macro):  0.4606


### Tại sao ResNet-18 tốt hơn GoogLeNet (khi huấn luyện "từ đầu")?

Mặc dù cả hai đều là mạng sâu, ResNet-18 có 2 "vũ khí" mà GoogLeNet-v1 (2014) không có:

1.  **Batch Normalization (BatchNorm):**
    * Đây là lý do **quan trọng nhất**. ResNet-18 có một lớp `BatchNorm` sau *mỗi* lớp `Conv`.
    * `BatchNorm` giúp "ổn định" dữ liệu giữa các lớp, cho phép mô hình hội tụ (học) **nhanh hơn và ổn định hơn rất nhiều**.
    * GoogLeNet-v1 ra đời *trước* BatchNorm (2015) nên nó không có, khiến việc huấn luyện từ đầu cực kỳ khó khăn.

2.  **Kết nối tắt (Residual Connections):**
    * Phép cộng `out += identity` giúp gradient (tín hiệu "học") chảy ngược về các lớp đầu tiên một cách dễ dàng. Nó giải quyết vấn đề "vanishing gradient" (gradient biến mất) mà các mạng rất sâu thường gặp phải.

---

### Hướng đi tiếp theo

Mặc dù 0.46 là tốt (so với 0.0079), nó vẫn chưa phải là kết quả tốt nhất. Lý do là chúng ta vẫn đang **huấn luyện từ đầu (from scratch)**.

Giải pháp để đạt kết quả cao hơn (ví dụ: F1 > 0.9) là sử dụng **Học Chuyển giao (Transfer Learning)**, bằng cách tải một mô hình ResNet-18 đã được huấn luyện trước (`pretrained=True`).